<a href="https://colab.research.google.com/github/Akshat-works/Text-Classifier/blob/main/Text_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [ ]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",split=('train[:60%]','train[60%:]', 'test'),as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.LJEVKG_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.LJEVKG_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.LJEVKG_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [ ]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape = [], dtype=tf.string,trainable=True)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(100), epochs=25,
                    validation_data=validation_data.batch(100), verbose=1)

Epoch 1/25
150/150 [==============================] - 7s 38ms/step - loss: 0.7159 - accuracy: 0.5994 - val_loss: 0.6157 - val_accuracy: 0.6570
Epoch 2/25
150/150 [==============================] - 4s 23ms/step - loss: 0.5430 - accuracy: 0.7196 - val_loss: 0.4820 - val_accuracy: 0.7551
Epoch 3/25
150/150 [==============================] - 5s 30ms/step - loss: 0.4097 - accuracy: 0.8089 - val_loss: 0.3881 - val_accuracy: 0.8200
Epoch 4/25
150/150 [==============================] - 4s 25ms/step - loss: 0.3214 - accuracy: 0.8601 - val_loss: 0.3557 - val_accuracy: 0.8199
Epoch 5/25
150/150 [==============================] - 4s 27ms/step - loss: 0.2648 - accuracy: 0.8895 - val_loss: 0.3162 - val_accuracy: 0.8604
Epoch 6/25
150/150 [==============================] - 5s 33ms/step - loss: 0.2250 - accuracy: 0.9093 - val_loss: 0.3095 - val_accuracy: 0.8715
Epoch 7/25
150/150 [==============================] - 5s 34ms/step - loss: 0.1939 - accuracy: 0.9272 - val_loss: 0.2978 - val_accuracy: 0.8744

In [ ]:
results = model.evaluate(test_data.batch(100), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

250/250 - 3s - loss: 0.6756 - accuracy: 0.8429 - 3s/epoch - 13ms/step
loss: 0.676
accuracy: 0.843
